In [1]:
import subprocess

# import subprocess; command = "yes | pip3.9 install psycopg2-binary --force-reinstall --no-cache-dir"; subprocess.run(command, shell=True)
result = subprocess.run(['pip3', 'install', 'ipython-autotime'], capture_output=True)
print(f"There was a problem installing the package.\nError output:\n{result.stderr.decode('utf-8')}") if result.returncode != 0 else print("Package installed successfully.")


Package installed successfully.


In [2]:
#Configuration
import os
import sys


#Python: Data Analysis
import pandas as pd




In [3]:
#Python: Notebook Configuration
from home_credit_risk import Configuration

#Python: Data Input/Output
from home_credit_risk import get_filename_path_list_from_filename_path_regular_expression
from home_credit_risk import get_filename_list

#PostgreSQL: Setup
from home_credit_risk import generate_parent_paths_from
from home_credit_risk import create_connection
from home_credit_risk import execute_query
from home_credit_risk import construct_query_from_table_name_column_name_list_and_data_type_list
from home_credit_risk import create_and_copy_file_to_postgresql_table
from home_credit_risk import extract_database_table_data_type_list_from_dataframe
from home_credit_risk import create_database_table_and_return_dataframe_from_single_filename
from home_credit_risk import create_database_table_and_return_dataframe_from_filename_regular_expression


#PostgreSQL: Data Analysis
from home_credit_risk import get_the_column_name_list_from_table_name
from home_credit_risk import get_the_table_column_values_and_value_counts
from home_credit_risk import get_the_table_column_data_types_and_value_count


#Python: Utility Function
from home_credit_risk import occupy_the_kernel

#Python: Data Analysis
from home_credit_risk import preview_df
from home_credit_risk import preview_list
from home_credit_risk import filter_this_data_frame_by_column_name_and_value_list




In [4]:
%load_ext autotime

%load_ext autoreload
%autoreload 2

time: 15.1 ms (started: 2024-04-11 23:55:17 -07:00)


In [5]:
skip_create_database_tables=True

configuration = Configuration(skip_create_database_tables=skip_create_database_tables)
configuration


time: 14.3 ms (started: 2024-04-11 23:55:17 -07:00)


### Connect to the Database 'postgres'

In [6]:
#Create the Connection to the Database 'postgres'
connection = create_connection(db_name="postgres", db_user="postgres", db_password="postgres", db_host="127.0.0.1", db_port="5432")

#Drop the Database 'database1' (if it Exists)
query="DROP DATABASE IF EXISTS database1;"
execute_query(connection=connection, query=query)

#Drop the Database 'postgres' (if it Exists)
query="DROP DATABASE IF EXISTS postgres;"
execute_query(connection=connection, query=query)

#Get the Database Names
query="SELECT datname FROM pg_database WHERE datistemplate = false;"
execute_query(connection=connection, query=query)


Connection to PostgreSQL DB successful
Query executed successfully
The error 'cannot drop the currently open database
' occurred
Query executed successfully


,0
0,postgres


time: 293 ms (started: 2024-04-11 23:55:17 -07:00)


### Get the Table Names of the Database 'postgres'

In [7]:
query="SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_type = 'BASE TABLE';"

result=execute_query(connection, query=query, table_name=None, column_name_list=None)
result


Query executed successfully


,0
0,train_debitcard_1
1,train_credit_bureau_b_2
2,train_person_1
3,train_deposit_1
4,train_static_0
5,train_static_cb_0
6,train_applprev_1
7,train_credit_bureau_a_1
8,train_credit_bureau_a_2
9,train_base


time: 43.5 ms (started: 2024-04-11 23:55:18 -07:00)


In [8]:
filename_star_list = ['train_static_0_*.csv', 'train_applprev_1_*.csv', 'train_credit_bureau_a_1_*.csv', 'train_credit_bureau_a_2_*.csv']
filename_single_list=['train_base.csv', 'train_static_cb_0.csv', 'train_tax_registry_a_1.csv', 'train_tax_registry_b_1.csv', 'train_tax_registry_c_1.csv', 'train_credit_bureau_b_1.csv', 'train_other_1.csv', 'train_person_1.csv', 'train_deposit_1.csv', 'train_debitcard_1.csv', 'train_credit_bureau_b_2.csv'] #[filename for filename in filename_list_with_regular_expressions if not filename in filename_star_list]
#filename_list_with_regular_expressions = ['train_base.csv', 'train_static_cb_0.csv', 'train_static_0_*.csv', 'train_applprev_1_*.csv', 'train_tax_registry_a_1.csv', 'train_tax_registry_b_1.csv', 'train_tax_registry_c_1.csv', 'train_credit_bureau_a_1_*.csv', 'train_credit_bureau_b_1.csv', 'train_other_1.csv', 'train_person_1.csv', 'train_deposit_1.csv', 'train_debitcard_1.csv', 'train_credit_bureau_b_2.csv', 'train_credit_bureau_a_2_*.csv']
filename_star_list.sort(); filename_single_list.sort()
preview_list(filename_star_list+filename_single_list)


15


['train_applprev_1_*.csv',
 'train_credit_bureau_a_1_*.csv',
 'train_credit_bureau_a_2_*.csv',
 'train_static_0_*.csv',
 'train_base.csv',
 'train_credit_bureau_b_1.csv',
 'train_credit_bureau_b_2.csv',
 'train_debitcard_1.csv',
 'train_deposit_1.csv',
 'train_other_1.csv',
 'train_person_1.csv',
 'train_static_cb_0.csv',
 'train_tax_registry_a_1.csv',
 'train_tax_registry_b_1.csv',
 'train_tax_registry_c_1.csv']

time: 13.2 ms (started: 2024-04-11 23:55:18 -07:00)


### Create a Database Table for Each .CSV Filename Collection

In [9]:
if configuration.skip_create_database_tables == False:
    for filename_regular_expression in filename_star_list:    
        _ = create_database_table_and_return_dataframe_from_filename_regular_expression(connection, filename_regular_expression, return_result=False)
else:
    pass

time: 10.6 ms (started: 2024-04-11 23:55:18 -07:00)


In [10]:
#Get Database Table Names
query="SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_type = 'BASE TABLE';"
result=execute_query(connection, query=query, table_name=None, column_name_list=None)
result


Query executed successfully


,0
0,train_debitcard_1
1,train_credit_bureau_b_2
2,train_person_1
3,train_deposit_1
4,train_static_0
5,train_static_cb_0
6,train_applprev_1
7,train_credit_bureau_a_1
8,train_credit_bureau_a_2
9,train_base


time: 13.2 ms (started: 2024-04-11 23:55:18 -07:00)


### Create a Database Table for Each Single .CSV File

In [11]:
if configuration.skip_create_database_tables == False:
    for filename in filename_single_list:
        create_database_table_and_return_dataframe_from_single_filename(connection=connection, filename=filename, return_result=False)
else:
    pass

time: 9.26 ms (started: 2024-04-11 23:55:18 -07:00)


In [12]:
#Get Database Table Names
query="SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_type = 'BASE TABLE';"
result=execute_query(connection, query=query, table_name=None, column_name_list=None)
result



Query executed successfully


,0
0,train_debitcard_1
1,train_credit_bureau_b_2
2,train_person_1
3,train_deposit_1
4,train_static_0
5,train_static_cb_0
6,train_applprev_1
7,train_credit_bureau_a_1
8,train_credit_bureau_a_2
9,train_base


time: 14.8 ms (started: 2024-04-11 23:55:18 -07:00)


### Appendix

In [13]:
# dir()
# %who_ls
# %who_ls function
# %who_ls str


time: 9.09 ms (started: 2024-04-11 23:55:18 -07:00)


In [14]:
# locals()
# globals()


time: 10.3 ms (started: 2024-04-11 23:55:18 -07:00)


In [15]:
# connection.close()


time: 9.45 ms (started: 2024-04-11 23:55:18 -07:00)


In [16]:
df_collection={}

key='feature_definitions'
relative_filename_path=os.path.join("..", "data", "raw", str(key)+".csv")
df_collection[key]=pd.read_csv(relative_filename_path)

print(df_collection[key].shape)
df_collection[key]



(465, 2)


,Variable,Description
0,actualdpd_943P,Days Past Due (DPD) of previous contract (actual).
1,actualdpdtolerance_344P,DPD of client with tolerance.
2,addres_district_368M,District of the person's address.
3,addres_role_871L,Role of person's address.
4,addres_zip_823M,Zip code of the address.
5,amount_1115A,Credit amount of the active contract provided by the credit bureau.
6,amount_416A,Deposit amount.
7,amount_4527230A,Tax deductions amount tracked by the government registry.
8,amount_4917619A,Tax deductions amount tracked by the government registry.
9,amtdebitincoming_4809443A,Incoming debit card transactions amount.


time: 38.2 ms (started: 2024-04-11 23:55:18 -07:00)


In [17]:


#in postgresql ordered table and replace the original table???

# -- Step 1: Create a new table with the desired order
# CREATE TABLE new_table_name (
#     -- Define columns similar to the original table
#     column1 datatype,
#     column2 datatype,
#     ...
# );

# -- Step 2: Copy data from the original table to the new table, ordering it as needed
# INSERT INTO new_table_name
# SELECT *
# FROM original_table_name
# ORDER BY column_to_order_by;

# -- Step 3: Drop the original table
# DROP TABLE original_table_name;

# -- Step 4: Rename the new table to match the original table's name
# ALTER TABLE new_table_name RENAME TO original_table_name;



time: 10.7 ms (started: 2024-04-11 23:55:18 -07:00)


In [18]:
# database table schema includes the column names, data types, constraints, and relationships with other tables.


time: 15.3 ms (started: 2024-04-11 23:55:18 -07:00)


In [19]:
# import sys
# variable_sizes = {name: sys.getsizeof(value) for name, value in locals().items()}
# for name, size in variable_sizes.items():
#     print(f"{name}: {size} bytes")


time: 13.5 ms (started: 2024-04-11 23:55:18 -07:00)


In [20]:
# import os
# def get_files_and_sizes(directory):
#     files_and_sizes = {}
#     for filename in os.listdir(directory):
#         filepath = os.path.join(directory, filename)
#         if os.path.isfile(filepath):
#             file_size_bytes = os.path.getsize(filepath)
#             file_size_mb = file_size_bytes / (1024 * 1024)
#             files_and_sizes[filename] = file_size_mb
#     return files_and_sizes

# files_and_sizes = get_files_and_sizes('/Users/pauljacob/MyDrive_pauljacob.datascience1@gmail.com/Projects/Home_Credit_Risk/data/raw/train/')
# for filename, size_mb in files_and_sizes.items():
#     print(f"{filename}: {size_mb:.2f} MB")



time: 10.6 ms (started: 2024-04-11 23:55:18 -07:00)


In [21]:
# ### Create and Copy into the Table 'train_applprev' the Data of 'train_applprev_1_0.csv' and 'train_applprev_1_1.csv'


# #Drop Tables
# table_name='feature_definitions, train_applprev_1_0, train_applprev_1_1, train_applprev'
# query = "DROP TABLE IF EXISTS {};".format(table_name)
# execute_query(connection, query=query, table_name=table_name, column_name_list=None)


# #Create Table 'train_applprev'
# table_name='train_applprev'
# column_name_list=['case_id', 'actualdpd_943P', 'annuity_853A', 'approvaldate_319D', 'byoccupationinc_3656910L', 'cancelreason_3545846M', 'childnum_21L', 'creationdate_885D', 'credacc_actualbalance_314A', 'credacc_credlmt_575A', 'credacc_maxhisbal_375A', 'credacc_minhisbal_90A', 'credacc_status_367L', 'credacc_transactions_402L', 'credamount_590A', 'credtype_587L', 'currdebt_94A', 'dateactivated_425D', 'district_544M', 'downpmt_134A', 'dtlastpmt_581D', 'dtlastpmtallstes_3545839D', 'education_1138M', 'employedfrom_700D', 'familystate_726L', 'firstnonzeroinstldate_307D', 'inittransactioncode_279L', 'isbidproduct_390L', 'isdebitcard_527L', 'mainoccupationinc_437A', 'maxdpdtolerance_577P', 'num_group1', 'outstandingdebt_522A', 'pmtnum_8L', 'postype_4733339M', 'profession_152M', 'rejectreason_755M', 'rejectreasonclient_4145042M', 'revolvingaccount_394A', 'status_219L', 'tenor_203L']
# data_type_list=['INTEGER',   'NUMERIC',        'NUMERIC',      'DATE',              'NUMERIC',                  'VARCHAR',               'NUMERIC',      'DATE',              'NUMERIC',                    'NUMERIC',              'NUMERIC',                'NUMERIC',               'VARCHAR',             'NUMERIC',                   'NUMERIC',         'VARCHAR',       'NUMERIC',      'DATE',               'VARCHAR',       'NUMERIC',      'DATE',           'DATE',                      'VARCHAR',         'DATE',              'VARCHAR',          'DATE',                       'VARCHAR',                  'BOOLEAN',           'BOOLEAN',          'NUMERIC',                'NUMERIC',              'INTEGER',    'NUMERIC',              'NUMERIC',   'VARCHAR',          'VARCHAR',         'VARCHAR',           'VARCHAR',                     'NUMERIC',               'VARCHAR',     'NUMERIC' ]
# #data_type_list=['VARCHAR(255)']+['NUMERIC']+  ['NUMERIC']+    ['DATE']+            ['NUMERIC']+                ['VARCHAR(255)']+        ['NUMERIC']+    ['DATE']+            ['NUMERIC']+                  ['NUMERIC']+            ['VARCHAR(255)']*(41-10)
# #                               ['SMALLINT']                                                                                            ['INTEGER']
# query = construct_query_from_table_name_column_name_list_and_data_type_list(table_name=table_name, column_name_list=column_name_list, data_type_list=data_type_list)
# execute_query(connection, query=query, table_name=table_name)


# #Copy in the Data 'train_applprev_1_0.csv'
# # Enable Read and Execute Permissions to the File "train_applprev_1_0.csv"
# filename_path = "/Users/pauljacob/MyDrive_pauljacob.datascience1@gmail.com/Projects/Home_Credit_Risk/data/raw/train/train_applprev_1_0.csv"
# filename_parent_paths=generate_parent_paths_from(filename_path=filename_path)
# command = "chmod a+rX {}".format(filename_parent_paths); subprocess.run(command, shell=True)

# #Copy Data into Table 'train_applprev'
# query = "copy {} FROM '{}' DELIMITER ',' CSV HEADER;".format(table_name, filename_path)
# execute_query(connection, query=query, table_name=table_name)



# #Copy in the Data 'train_applprev_1_1.csv'
# # Enable Read and Execute Permissions to the File "train_applprev_1_0.csv"
# filename_path = "/Users/pauljacob/MyDrive_pauljacob.datascience1@gmail.com/Projects/Home_Credit_Risk/data/raw/train/train_applprev_1_1.csv"
# filename_parent_paths=generate_parent_paths_from(filename_path=filename_path)
# command = "chmod a+rX {}".format(filename_parent_paths); subprocess.run(command, shell=True)

# #Copy Data into Table 'train_applprev'
# query = "copy {} FROM '{}' DELIMITER ',' CSV HEADER;".format(table_name, filename_path)
# execute_query(connection, query=query, table_name=table_name) #Add efficiency? #Source: https://hakibenita.com/fast-load-data-python-postgresql#benchmark
# #time: 32 s (started: 2024-04-09 19:00:31 -07:00)





# ###  Alter Type to SMALLINT / INTEGER from NUMERIC


# ###  Alter Type to SMALLINT from NUMERIC: Column Name 'actualdpd_943P', i.e. Days Past Due (DPD) of previous contract (actual).
# table_name='train_applprev'
# column_name='actualdpd_943P'
# data_type='SMALLINT'
# query="ALTER TABLE {} ALTER COLUMN {} TYPE {} USING {}::{};".format(table_name, column_name, data_type, column_name, data_type)
# execute_query(connection, query=query, table_name=table_name, column_name_list=[column_name])


# ### Alter Type to INTEGER from NUMERIC: Column Name 'childnum_21L', i.e. Number of children in the previous application.
# column_name='childnum_21L'
# table_name='train_applprev'
# data_type='INTEGER'
# query="ALTER TABLE {} ALTER COLUMN {} TYPE {} USING ROUND({}::NUMERIC, 0);".format(table_name, column_name, data_type, column_name)
# execute_query(connection=connection, query=query, table_name=table_name, column_name_list=[column_name])

# #time: 1min 1s (started: 2024-04-09 19:01:03 -07:00)

time: 11.8 ms (started: 2024-04-11 23:55:18 -07:00)
